In [1]:
!pip install -U fsspec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.5.1 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64

In [2]:
!pip install -U datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [3]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00


- Selecciona un modelo pre-entrenado como base y realiza *fine-tuning* para resolver alguna tarea de NLP que te parezca reelevante
  - Procura utilizar datasets pequeños para que sea viable
  - Recuerda las posibles tareas disponibles en HF `*For<task>`
- Desarrolla y pon en producción un prototipo del modelo
  - Incluye una URL pública donde podamos ver tu proyecto
  - Recomendamos usar framewoks de prototipado (*streamlit* o *gradio*) y el *free-tier* de *spaces* de hugging face
    - https://huggingface.co/spaces/launch
    - https://huggingface.co/docs/hub/spaces-sdks-streamlit
    - https://huggingface.co/docs/hub/spaces-sdks-gradio
- Reporta que tan bien se resolvió la tarea y que tan útil fue tu app
- Reporta retos y dificultades al realizar el *fine-tuning* y al poner tu modelo en producción

## Extra

- Utiliza [code carbon](https://codecarbon.io/#howitwork) para reportar las emisiones de tu app

In [4]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import numpy as np

# Cargar dataset y tokenizer
dataset = load_dataset('imdb')
small_train = dataset['train'].shuffle(seed=42).select(range(1000))
small_test = dataset['test'].shuffle(seed=42).select(range(200))

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [5]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [6]:
# Tokenización
# Esta función toma un diccionario de ejemplos (donde 'text' es la clave de los textos de entrada)
# y aplica el tokenizador para convertir cada texto en una secuencia de tokens entendible por el modelo.
# Se utiliza padding para que todas las secuencias tengan la misma longitud (max_length),
# y truncation para cortar los textos que excedan la longitud máxima permitida por el modelo.
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Se aplica la función de tokenización a los datasets de entrenamiento y prueba.
# El parámetro batched=True permite procesar múltiples ejemplos a la vez, lo cual acelera el procesamiento.
train_dataset = small_train.map(tokenize_function, batched=True)
test_dataset = small_test.map(tokenize_function, batched=True)

# Métrica
# Se carga la métrica de precisión (accuracy) utilizando la biblioteca `evaluate` de Hugging Face.
metric = evaluate.load("accuracy")

# Esta función se usa para calcular métricas de evaluación durante el entrenamiento.
# Toma las predicciones del modelo (logits) y las etiquetas verdaderas,
# convierte los logits en etiquetas predichas usando argmax, y luego calcula la precisión.
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Modelo
# Se carga un modelo preentrenado de clasificación de texto basado en DistilBERT.
# 'distilbert-base-uncased' es una versión ligera de BERT que no distingue entre mayúsculas y minúsculas.
# El parámetro num_labels=2 indica que es una tarea de clasificación binaria.
# Finalmente, el modelo se mueve al dispositivo adecuado (CPU o GPU) para entrenamiento o inferencia.
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2
).to(device)  # Mover modelo a GPU


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # Ajustar según memoria GPU
    per_device_eval_batch_size=32,   # Puede ser mayor para evaluación
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    report_to="none",
    fp16=True,  # Activar mixed-precision para mayor velocidad en GPUs modernas
    gradient_accumulation_steps=2,  # Útil si el batch size máximo es pequeño
)

# 6. Entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


In [8]:
# Fine-tuning
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.595100,0.534211,0.795000
2,0.309900,0.362265,0.860000
3,0.239100,0.351747,0.845000


TrainOutput(global_step=96, training_loss=0.4188417891661326, metrics={'train_runtime': 59.1512, 'train_samples_per_second': 50.717, 'train_steps_per_second': 1.623, 'total_flos': 397402195968000.0, 'train_loss': 0.4188417891661326, 'epoch': 3.0})

In [9]:

model.save_pretrained('./sentiment_model')
tokenizer.save_pretrained('./sentiment_model')

# Verificar que existen
!ls ./sentiment_model

config.json	   special_tokens_map.json  tokenizer.json
model.safetensors  tokenizer_config.json    vocab.txt


In [10]:
from google.colab import files
!zip -r sentiment_model.zip sentiment_model/
files.download('sentiment_model.zip')

  adding: sentiment_model/ (stored 0%)
  adding: sentiment_model/tokenizer.json (deflated 71%)
  adding: sentiment_model/special_tokens_map.json (deflated 42%)
  adding: sentiment_model/tokenizer_config.json (deflated 75%)
  adding: sentiment_model/model.safetensors (deflated 8%)
  adding: sentiment_model/vocab.txt (deflated 53%)
  adding: sentiment_model/config.json (deflated 45%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>